In [ ]:
# %matplotlib inline
# import matplotlib.pyplot as plt
# import matplotlib.image as mpimg

In [304]:
%load_ext line_profiler

The line_profiler extension is already loaded. To reload it, use:
  %reload_ext line_profiler


In [305]:
import numpy as np
import cv2

In [306]:
class MOG(object):
  def __init__(self,frame, K=3,T=0.3,lr=0.25):
    self.rows = frame.shape[0]
    self.cols = frame.shape[1]
    self.N = frame.shape[2]
    self.K = K
    self.T = T
    self.mean = np.zeros((self.rows,self.cols,K,self.N),dtype=np.float32)
    self.std_dev = np.ones((self.rows,self.cols,K),dtype=np.float32)
    self.weight = np.ones((self.rows,self.cols,K),dtype=np.float32)/K
    self.dist_mult = 2.5
    self.lr = lr
    self.HIGH_VAR=10
    self.denom_const = np.power(2*np.pi,(self.N)/2)

  def apply(self,frame):
    ## calculating mahalanaobis distance for each pixel
    sort_weights = self.weight/self.std_dev
    index_sort_weights = np.argsort(-sort_weights,axis=2)

    sqdist_frames = (np.sum((np.expand_dims(frame,axis=2)-self.mean)**2,axis=3))/self.std_dev
    prob_frames = np.exp((-0.5)*sqdist_frames)/(self.denom_const*(np.sqrt(self.N)*self.std_dev))  
    dist_frames = np.sqrt(sqdist_frames)
    admissible_frames = dist_frames < self.dist_mult*self.std_dev
    
    lin_indx = np.reshape(index_sort_weights,[-1])
    row_indx = np.repeat(np.arange(self.rows),self.cols*self.K)
    col_indx = np.tile(np.repeat(np.arange(self.cols),self.K),self.rows)
    sorted_wgts = np.reshape(self.weight[row_indx,col_indx,lin_indx],(self.rows,self.cols,-1))
    background_dist = np.cumsum(sorted_wgts,axis=2)<self.T
    is_background = np.any(np.logical_and(background_dist,admissible_frames),axis=2)
    result = (np.logical_not(is_background)*255).astype(np.uint8)
    
    modify_mask = np.any(admissible_frames,axis=2)
    matched_gaussian = np.choose(np.argmax(admissible_frames,axis=2),index_sort_weights.transpose(2,0,1))
    
    self.weight -= (np.expand_dims(modify_mask,axis=2)*self.weight)*(self.lr)
    row_indx = np.repeat(np.arange(self.rows),self.cols)
    col_indx = np.tile((np.arange(self.cols)),self.rows)
    lin_indx = np.reshape(matched_gaussian,[-1])
    self.weight[row_indx,col_indx,lin_indx] = np.reshape(modify_mask,[-1])*self.lr
    
    rho = np.reshape(modify_mask,[-1])*self.lr*(prob_frames[row_indx,col_indx,lin_indx] )
    self.mean[row_indx,col_indx,lin_indx,:] -= np.expand_dims(rho,axis=2)*self.mean[row_indx,col_indx,lin_indx,:]
    self.mean[row_indx,col_indx,lin_indx,:] += np.expand_dims(rho,axis=2)*frame[row_indx,col_indx,:]
    
    prev_var = (self.std_dev[row_indx,col_indx,lin_indx]*np.reshape(modify_mask,[-1]))**2
    add_term = rho*np.sum((frame[row_indx,col_indx,:]-self.mean[row_indx,col_indx,lin_indx,:])**2,axis=1)
    new_var = (1-rho)*prev_var + add_term
    self.std_dev[row_indx,col_indx,lin_indx] = np.sqrt(new_var)
    
    
    ## Adding new gaussian
    last_idx = index_sort_weights[:,:,-1]
    lin_indx = np.reshape(last_idx,[-1])
    updation_mask = np.logical_not(modify_mask)
    new_mean_values = (self.mean - (np.expand_dims(frame,axis=2)))[row_indx,col_indx,lin_indx,:]
    lin_mask = np.reshape(updation_mask,[-1])
    self.mean[row_indx,col_indx,lin_indx,:] -= np.expand_dims(lin_mask,axis=3)*new_mean_values
    self.std_dev[row_indx,col_indx,lin_indx] += lin_mask*self.HIGH_VAR ## CHECK THIS
#     result = np.zeros(frame.shape[:2],dtype=np.uint8)
#     for i in range(frame.shape[0]):
#       for j in range(frame.shape[1]):
#         indices = index_sort_weights[i][j]
#         values = self.weight[indices,i,j]
#         background_dist = np.cumsum(values)<self.T
#         admissible = admissible_frames[i][j]
#         is_background = np.logical_and(background_dist,admissible)
#         idx = np.argmax(is_background)
#         if (is_background[idx]):
#           result[i][j] = 255

#         if (not (np.any(admissible))):
#           #ADD NEW GAUSSIAN
#           idx = indices[-1]
#           self.mean[idx][i][j] = frame[i][j]
#           self.std_dev[idx][i][j] = self.HIGH_VAR
#         else:
#           rho = self.lr*(prob_frames[idx][i][j])

#           self.mean[idx][i][j] *= (1-rho)
#           self.mean[idx][i][j] += frame[i][j]*rho
#           variance = (1-rho)*(self.std_dev[idx][i][j]**2) + rho*(np.sum((frame[i][j]-self.mean[idx][i][j])**2))
#           self.std_dev[idx][i][j] = np.sqrt(variance)
    return result



In [307]:
THRESHOLD = 30
CONFIDENCE = 0.1

In [308]:
# cap = cv2.VideoCapture('./videos/vtest.avi')

In [314]:
cap = cv2.VideoCapture(0)

In [315]:
#fgbg = cv2.createBackgroundSubtractorMOG2()

In [316]:
ret, frame = cap.read()
background = np.zeros(frame.shape)
# frame = cv2.resize(frame, (100, 50))

In [317]:
fgbg = MOG(frame)

In [318]:
while(1):
    ret, frame = cap.read()
#     frame = cv2.resize(frame, (100, 50)) 
    if (not ret):
        break
    fgmask = fgbg.apply(frame)
    cv2.imshow('frame',frame)
    cv2.imshow('output',fgmask)
    
#     bgmask = (fgmask<THRESHOLD)[:,:,None]
#     fgmask = (fgmask>(128 - THRESHOLD))[:,:,None]
#     new_bg = frame*bgmask
#     new_fg = frame*fgmask
#     bg_prob_mask = np.ones(frame.shape) - (bgmask*CONFIDENCE)
#     background = np.asarray((bg_prob_mask*background + (1-bg_prob_mask)*new_bg),np.uint8)
#     foreground = np.asarray(fgmask*new_fg,np.uint8)
#     cv2.imshow('background',background)
#     cv2.imshow('foreground',foreground)
    
    k = cv2.waitKey(30) & 0xff
    if k == 27:
        break
cap.release()
cv2.destroyAllWindows()

/home/suyash1212/.local/lib/python3.4/site-packages/ipykernel_launcher.py:44: DeprecationWarning: Both axis > a.ndim and axis < -a.ndim - 1 are deprecated and will raise an AxisError in the future.
/home/suyash1212/.local/lib/python3.4/site-packages/ipykernel_launcher.py:45: DeprecationWarning: Both axis > a.ndim and axis < -a.ndim - 1 are deprecated and will raise an AxisError in the future.
/home/suyash1212/.local/lib/python3.4/site-packages/ipykernel_launcher.py:59: DeprecationWarning: Both axis > a.ndim and axis < -a.ndim - 1 are deprecated and will raise an AxisError in the future.
/home/suyash1212/.local/lib/python3.4/site-packages/ipykernel_launcher.py:18: RuntimeWarning: invalid value encountered in true_divide
/home/suyash1212/.local/lib/python3.4/site-packages/ipykernel_launcher.py:21: RuntimeWarning: divide by zero encountered in true_divide
/home/suyash1212/.local/lib/python3.4/site-packages/ipykernel_launcher.py:22: RuntimeWarning: invalid value encountered in true_divide


In [ ]:
np.

In [ ]:
params = fgbg.params[30][50]

In [ ]:
params

In [ ]:
[p["weight"] for p in params]

In [ ]:
x = np.zeros(frame.shape[:2],dtype=np.uint8)

In [ ]:
x

In [ ]:
x[0][0]

In [ ]:
x = np.array([3,4,5])

In [ ]:
y = np.array([1,0,3])

In [ ]:
z = np.multiply(x,y)

In [ ]:
np.argmax(z)

In [ ]:
z

In [83]:
np.argmax(x>6)

NameError: name 'x' is not defined

In [ ]:






o

### TESTING

In [192]:
frame = np.reshape([i+8 for i in range(24)],(3,4,2))

In [193]:
fgbg = MOG(frame)

In [194]:
self = fgbg

In [226]:
a1 = np.reshape([i+4 for i in range(24)],(3,4,2))
a2 = np.reshape([i+6 for i in range(24)],(3,4,2))
a3 = np.stack([a1,a2],axis=3)

In [227]:
self.mean = a3

In [228]:
self.std_dev = np.reshape([1.5]*24,(3,4,2))

In [229]:
self.weight = np.reshape([0.3,0.7]*12,(3,4,2))

In [230]:
sort_weights = self.weight/self.std_dev

In [231]:
index_sort_weights = np.argsort(-sort_weights,axis=2)

In [232]:
sqdist_frames = np.sum((np.expand_dims(frame,axis=2)-self.mean)**2,axis=3)/self.std_dev

In [233]:
dist_frames = np.sqrt(sqdist_frames)

In [234]:
prob_frames = np.exp(-0.5*sqdist_frames)/(self.denom_const*(np.sqrt(self.N)*self.std_dev))

In [237]:
admissible_frames = dist_frames < self.dist_mult*self.std_dev

In [239]:
lin_indx = np.reshape(index_sort_weights,[-1])

In [241]:
row_indx = np.repeat(np.arange(self.rows),self.cols*self.K)
col_indx = np.tile(np.repeat(np.arange(self.cols),self.K),self.rows)

In [242]:
sorted_wgts = np.reshape(self.weight[row_indx,col_indx,lin_indx],(self.rows,self.cols,-1))

In [245]:
self.weight[row_indx,col_indx,lin_indx]

array([ 0.7,  0.3,  0.7,  0.3,  0.7,  0.3,  0.7,  0.3,  0.7,  0.3,  0.7,
        0.3,  0.7,  0.3,  0.7,  0.3,  0.7,  0.3,  0.7,  0.3,  0.7,  0.3,
        0.7,  0.3])

In [247]:
sort_weights

array([[[ 0.2       ,  0.46666667],
        [ 0.2       ,  0.46666667],
        [ 0.2       ,  0.46666667],
        [ 0.2       ,  0.46666667]],

       [[ 0.2       ,  0.46666667],
        [ 0.2       ,  0.46666667],
        [ 0.2       ,  0.46666667],
        [ 0.2       ,  0.46666667]],

       [[ 0.2       ,  0.46666667],
        [ 0.2       ,  0.46666667],
        [ 0.2       ,  0.46666667],
        [ 0.2       ,  0.46666667]]])

In [249]:
background_dist = np.cumsum(sorted_wgts,axis=2)<self.T

In [251]:
is_background = np.any(np.logical_and(background_dist,admissible_frames),axis=2)

array([[False, False, False, False],
       [False, False, False, False],
       [False, False, False, False]], dtype=bool)